# Create the input features and output lables for the Machine Learning classification training

## Premise of the experiment
The question I am trying to answer is whether you can predict a successfull city based solely on the information available in the 7, 15 in the case of Russia, tiles immediately around the settled plot. In order to do this we need to define what a successful city looks like, and then how to use the plot information available as input to the classification ML engine.

As I am the creator of the dataset I have much more freedom than one would normally have in any ML competition.

## Determine the label (aka value to predict)
I decided at the start to limit the data capture to the first 50 turns after a city is settled. Also, only using cities settled in the first 3 turns of the game to reduce variability. For example, later in the game strategic resources are show that cleary has impact we can't account for.

As cities can, and relatively frequently get settled on the second turn the first task is to standardise and align the "turns". This is done with the turns column in the **cityPerTurnView** table. All I need to do here is extract it.

Based on the shape of *Faith* yields I decided to leave it out. That means the yields of interest are: food, production, gold, science, and culture. Science and culture get most of their yields from buildings and improvements by I left them in as they are key yields.

To determine the "good" cities I used the total yield a city produced for the first 50 turns and assigned a score from 0 through 9 based on deciles (10 quantiles) of that specific yield. I then summed the 5 scores (1 each for food, production, gold, science, and culture) and allocated this as the city's score. The maximum score a city could get is 45 and the least is 0. I then marked all cities in the top 25% (top quartile) of this cityTotal as being good and all the rest as poor.

## Determine the features (aka values used as input to predict)
In the end I decided to use all 19 tiles (plots) that are within 2 tiles from the city centre. By turn 50 many of the 2nd ring tiles are used and as Russia starts with 15 or so available it seemed the best approach to give slightly more information to the model.

I also combined terrain and feature into one category as feature never exists in isolation. However resources (bonus, luxury, and strategic) formed a different category. When looking at the resources there were a few that occured frequently and early and some, especially luxury, resource infrequently.

All luxuries were grouped in Lux. All bonus resources were grouped in OtherBonus - with the exception of Wheat, Stone, Cattle, and Rice. There is only one strategic resource available so I left Horses as is too.

Lastly, I categorised by percentage. For example, I count all the PlainsHills in a city and then divide by 19 to determine the percentage of tiles with PlainsHills. This improved model classification vs. supplying the counts.

## Comments regarding the decisions
Keep in mind that these decisions were not made in one linear sequence. I did not know the best way to do this when I started and this is the result of much experimentation. Also, I am more that happy to receive feedback on these decisions as I sure this could be improved.

This workbook produces the features and labels CSV files that I use in the next one to create a classification model.


In [1]:
import sqlite3
import pandas as pd

In [2]:
cnx = sqlite3.connect('Database/Civ6CitySettledData.db')
cur = cnx.cursor()
#print(cnx)
#print(cur)

## Labels:

Use the specifically created database view to retrieve the per turn data collected. This view also aligns timelines, that is, takes care of cities settled in turn 2, or even turn 3. I was surprised to learn the AI actually moves settlers before settling.

In [3]:
sqlSelect = 'SELECT * FROM cityPerTurnView WHERE turns >= 1 and turns <= 50'
cityPt = pd.read_sql_query(sqlSelect, cnx)
print(cityPt.shape)

(25100, 29)


I have excluded Faith as it was simply too variable to use. Also, it doesn't appear to be a core yield in general, although it is situationally very useful.

In [4]:
yields = ['food', 'production', 'gold', 'science', 'culture']
cityIds = list(cityPt['cityId'].unique())
labelsDf = pd.DataFrame(columns={'foodTotal', 'foodScore', 'productionTotal', 'productionScore',
                                 'goldTotal', 'goldScore', 'scienceTotal', 'scienceScore',
                                 'cultureTotal', 'cultureScore', 'cityTotal', 'cityScore'},
                        index=cityIds)
labelsDf.reset_index(level=0,inplace=True)
labelsDf.rename(columns={'index':'cityId'}, inplace=True)
labelsDf.sort_values(by='cityId', inplace=True)
labelsDf.index = pd.RangeIndex(len(labelsDf.index))
labelsDf.fillna(0, inplace=True)

# As I decided to use binary classification we have two steps here, internally
# I use deciles to determine the cityScore, then I replace all with the 2 quantile
# external one. (top 25% is good, rest "poor")
quantileInternal = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]
quantileExternal = [0, .75, 1]

for yld in yields:
    columnName = "{}PerTurn".format(yld)
    cumulativeDf = cityPt[['cityId', 'turns', columnName]].pivot(index='turns',
                                                                 columns='cityId',
                                                                 values=columnName).cumsum()
    t50 = cumulativeDf.loc[50].to_frame()
    t50.reset_index(level=0, inplace=True)
    t50.rename(columns={50:"{}Total".format(yld)}, inplace=True)
    # join here via boolean map as using cityID, not index - see df.update below
    labelsDf.loc[labelsDf.cityId.isin(t50.cityId), "{}Total".format(yld)] = t50["{}Total".format(yld)]
    
    quantilesDf = pd.qcut(labelsDf["{}Total".format(yld)], quantileInternal, labels=False).to_frame()
    quantilesDf.rename(columns={"{}Total".format(yld):"{}Score".format(yld)}, inplace=True)
    # update works cause we have index alignment
    labelsDf.update(quantilesDf)
    labelsDf['cityTotal'] = labelsDf['cityTotal'] + labelsDf["{}Score".format(yld)]

quantilesDf = pd.qcut(labelsDf['cityTotal'], quantileExternal, labels=False).to_frame()
quantilesDf.rename(columns={'cityTotal':'cityScore'}, inplace=True)
# update works cause we have index alignment
labelsDf.update(quantilesDf)

for yld in yields:
    labelsDf['{}Score'.format(yld)] = pd.qcut(labelsDf['{}Total'.format(yld)], quantileExternal, labels=False)

print(labelsDf.head())

   cityId  cultureScore  goldTotal  productionTotal  goldScore  scienceTotal  \
0       1             0      289.8           492.45          0        182.68   
1       2             0      660.7           490.85          1        196.94   
2       3             0      251.5           159.15          0        180.63   
3       4             0      262.5           157.30          0        150.22   
4       5             0      250.0           105.40          0        185.06   

   cityScore  cultureTotal  productionScore  foodScore  cityTotal  foodTotal  \
0          0        101.97                1          0         17      268.0   
1          1        164.75                0          0         32      321.0   
2          0        153.38                0          0         18      469.0   
3          0        140.42                0          0          6      195.0   
4          0        141.82                0          1         15      500.0   

   scienceScore  
0             0  
1 

In [5]:
# Forgetting encoding='utf-8' here causes much pain later!
labelsDf.to_csv('ModelInput/labels.csv', encoding='utf-8', index=False)

## Features:

Use the cityPlotsSettled data collected to prepare the features we intend using as input into the model. The category key idea is explained in the introduction.

We also specify the cityHasRiver as 1 = True, and 0 = False.

In [6]:
# Decision about what to include and what now...
sqlSelect = 'SELECT * FROM cityPlotsSettled'
cityPs = pd.read_sql_query(sqlSelect, cnx)
print(cityPs.shape)
#print(cityPs.dtypes)

(9538, 15)


### Extract the categorical features

In [7]:
#Create two new categorical columns in the sourceDf first

# Terrain and Feature categories - these never occur in isolation so group them!
cityPs['catTf'] = cityPs['terrain'] + cityPs['feature']
# Remove None, whitespace, and brackets
cityPs['catTf'] = cityPs['catTf'].apply(lambda s: s.replace('None', '').replace(' and ', '').replace(' ', ''))
cityPs['catTf'] = cityPs['catTf'].apply(lambda s: s.replace('(','').replace(')', ''))

# And, a category key for resources...
cityPs['catR'] = cityPs['resource']
# Remove all "hidden" strategic resources - Only Horses are left
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Uranium', 'None').replace('Oil', 'None').replace('Niter', 'None'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Aluminum', 'None').replace('Coal', 'None').replace('Iron', 'None'))
# Consolidate Luxuries
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Dyes', 'Lux').replace('Silver', 'Lux').replace('Diamonds', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Tea', 'Lux').replace('Salt', 'Lux').replace('Olives', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Ivory', 'Lux').replace('Sugar', 'Lux').replace('Coffee', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Cotton', 'Lux').replace('Furs', 'Lux').replace('Whales', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Marble', 'Lux').replace('Jade', 'Lux').replace('Turtles', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Gypsum', 'Lux').replace('Mercury', 'Lux').replace('Tobacco', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Wine', 'Lux').replace('Truffles', 'Lux').replace('Incense', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Silk', 'Lux').replace('Citrus', 'Lux').replace('Spices', 'Lux'))
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Cocoa', 'Lux').replace('Pearls', 'Lux').replace('Amber', 'Lux'))
## Consolidate bonus resources
#cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Sheep', 'OtherBonus')) #.replace('Bananas', 'OtherBonus'))
#cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Fish', 'OtherBonus').replace('Deer', 'OtherBonus'))
#cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace('Crabs', 'OtherBonus').replace('Copper', 'OtherBonus'))
# Remove None, whitespace, and brackets
cityPs['catR'] = cityPs['catR'].apply(lambda s: s.replace(' ', '').replace('(','').replace(')', ''))

# Still have 35 different "categories" here...
#print(cityPs[['plotId','catTf']].groupby('catTf').count().sort_values(by='catTf', ascending=True).count())
#print(cityPs[['plotId','catR']].groupby('catR').count().sort_values(by='catR', ascending=True).count())

print("\nTerrain Features categories:")
print(list(cityPs['catTf'].unique()))
print("\nResource categories:")
print(list(cityPs['catR'].unique()))

# Here we effectively "One Hot Encode" the categories, that is turn the categorical features into
# numerical ones

# Create the destination DataFrame using the list of categories.

cols = ['cityId'] + list(cityPs['catTf'].unique()) + list(cityPs['catR'].unique())
#print(cols, "\n", type(cols))
#list(filter(None, list(cityPs['catR'].unique())))
featuresDf = pd.DataFrame(columns = cols)
featuresDf = featuresDf.astype('float')
print('\nFeatures DataFrame will contain:')
print(featuresDf.shape)
print(featuresDf.columns)


Terrain Features categories:
['PlainsHills', 'PlainsMountain', 'PlainsRainforest', 'Plains', 'PlainsHillsWoods', 'DesertHills', 'PlainsWoods', 'PlainsHillsRainforest', 'Grassland', 'CoastLake', 'GrasslandHills', 'DesertMountain', 'Desert', 'GrasslandWoods', 'GrasslandMarsh', 'Ocean', 'Tundra', 'Snow', 'GrasslandHillsWoods', 'GrasslandMountain', 'CoastLakeReef', 'DesertFloodplains', 'TundraWoods', 'TundraMountain', 'TundraHills', 'DesertOasis', 'TundraHillsWoods']

Resource categories:
['None', 'Sheep', 'Lux', 'Wheat', 'Bananas', 'Fish', 'Stone', 'Cattle', 'Horses', 'Deer', 'Rice', 'Crabs', 'Copper']

Features DataFrame will contain:
(0, 41)
Index(['cityId', 'PlainsHills', 'PlainsMountain', 'PlainsRainforest', 'Plains',
       'PlainsHillsWoods', 'DesertHills', 'PlainsWoods',
       'PlainsHillsRainforest', 'Grassland', 'CoastLake', 'GrasslandHills',
       'DesertMountain', 'Desert', 'GrasslandWoods', 'GrasslandMarsh', 'Ocean',
       'Tundra', 'Snow', 'GrasslandHillsWoods', 'Grasslan

### Transform raw plot data into model input features

In [8]:
# Loop through all plots, by city
for cityId in cityPs['recordedCityId'].unique():
    # Select all the plots in the city
    plotsDf = cityPs[cityPs['recordedCityId'] == cityId]

    # Very handy pandas functionality to count and percentage by category
    catTfSeries = round(plotsDf.groupby(['catTf'])['plotId'].count() / 19, 4)
    catRSeries = round(plotsDf.groupby(['catR'])['plotId'].count() / 19, 4)
#    catTfSeries = plotsDf.groupby(['catTf'])['plotId'].count()
#    catRSeries = plotsDf.groupby(['catR'])['plotId'].count()

    # loop through each series and add calculated values to the featuresDF
    featuresDf.loc[cityId, 'cityId'] = cityId
    for cat, val in catTfSeries.iteritems():
        featuresDf.loc[cityId, cat] = val
    for cat, val in catRSeries.iteritems():
        featuresDf.loc[cityId, cat] = val

# make sure NaN or empty cells are replaced with zero
featuresDf.fillna(0.0, inplace=True)
print(featuresDf.head())

   cityId  PlainsHills  PlainsMountain  PlainsRainforest  Plains  \
1     1.0       0.1579          0.0526            0.0526  0.3158   
2     2.0       0.0526          0.0000            0.1579  0.1053   
3     3.0       0.0526          0.0000            0.0526  0.3158   
4     4.0       0.0526          0.0000            0.1053  0.2105   
5     5.0       0.0000          0.0000            0.0526  0.0000   

   PlainsHillsWoods  DesertHills  PlainsWoods  PlainsHillsRainforest  \
1            0.1053       0.0526       0.1579                 0.0526   
2            0.0526       0.1053       0.1053                 0.0000   
3            0.0000       0.0000       0.1053                 0.0000   
4            0.0526       0.0000       0.0526                 0.0000   
5            0.0000       0.0000       0.0000                 0.0000   

   Grassland   ...     Wheat  Bananas    Fish   Stone  Cattle  Horses    Deer  \
1     0.0526   ...    0.0526   0.0000  0.0000  0.0000  0.0000  0.0000  0.0000

In [9]:
# cityHasRiver calculation...
for cityId in cityPs['recordedCityId'].unique():
    plotsDf = cityPs[cityPs['recordedCityId'] == cityId]
    
    featuresDf.loc[cityId, 'cityHasRiver'] = plotsDf[plotsDf['isCity'] == True].iloc[0,:].hasRiver

# Drop the "none" now
del featuresDf['None']

# Make sure all the dtypes are correct. All should be float except cityId and cityHasRiver
#featuresDf = featuresDf.astype('int')
featuresDf = featuresDf.astype('float')
featuresDf['cityId'] = featuresDf['cityId'].astype('int')
featuresDf['cityHasRiver'] = featuresDf['cityHasRiver'].astype('int')

In [10]:
print(featuresDf.head())
print(featuresDf.dtypes)

   cityId  PlainsHills  PlainsMountain  PlainsRainforest  Plains  \
1       1       0.1579          0.0526            0.0526  0.3158   
2       2       0.0526          0.0000            0.1579  0.1053   
3       3       0.0526          0.0000            0.0526  0.3158   
4       4       0.0526          0.0000            0.1053  0.2105   
5       5       0.0000          0.0000            0.0526  0.0000   

   PlainsHillsWoods  DesertHills  PlainsWoods  PlainsHillsRainforest  \
1            0.1053       0.0526       0.1579                 0.0526   
2            0.0526       0.1053       0.1053                 0.0000   
3            0.0000       0.0000       0.1053                 0.0000   
4            0.0526       0.0000       0.0526                 0.0000   
5            0.0000       0.0000       0.0000                 0.0000   

   Grassland      ...       Bananas    Fish   Stone  Cattle  Horses    Deer  \
1     0.0526      ...        0.0000  0.0000  0.0000  0.0000  0.0000  0.0000   


In [11]:
# Bad things happen downstream if you forget the encoding ... don't forget!
featuresDf.to_csv('ModelInput/features.csv', encoding='utf-8', index=False)